In [ ]:
PROJECT_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Project"
DATASET_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Project/data"
%cd "{PROJECT_PATH}"

/content/drive/My Drive/Deep_Learning_Assignments/Project


In [ ]:
# !nvidia-smi
!pip install timm>/tmp/xxy

# Convert to jpg

In [ ]:
# !sudo apt install imagemagick
# # /content/drive/My Drive/Deep_Learning_Assignments/Project
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/test/"
# !mogrify -format jpg *.jfif
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/healthy_wheat" 
# !mogrify -format jpg *.jfif  
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/leaf_rust"
# !mogrify -format jpg *.jfif  
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/stem_rust" 
# !mogrify -format jpg *.jfif 

# Fastai

In [ ]:
from fastai import * 
from fastai.vision import *

In [ ]:
# https://docs.fast.ai/vision.data.html#ImageDataBunch.from_folder
data = ImageDataBunch.from_folder(DATASET_PATH, train="train/", test='test/', valid_pct=0.15,
        ds_tfms=get_transforms(flip_vert=True), size=224, num_workers=4, seed=42, bs=32).normalize(imagenet_stats)
# print(data.classes)

In [ ]:
# print(data)

ImageDataBunch;

Train: LabelList (746 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
healthy_wheat,healthy_wheat,healthy_wheat,healthy_wheat,healthy_wheat
Path: /content/drive/My Drive/Deep_Learning_Assignments/Project/data;

Valid: LabelList (131 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
stem_rust,stem_rust,leaf_rust,leaf_rust,leaf_rust
Path: /content/drive/My Drive/Deep_Learning_Assignments/Project/data;

Test: LabelList (610 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: EmptyLabelList
,,,,
Path: /content/drive/My Drive/Deep_Learning_Assignments/Project/data


In [ ]:
# data.show_batch(rows=3, figsize=(15, 15)) 

In [ ]:
# https://docs.fast.ai/vision.models.html
# resnet18, resnet34, resnet50, resnet101, resnet152
# squeezenet1_0, squeezenet1_1
# densenet121, densenet169, densenet201, densenet161
# vgg16_bn, vgg19_bn
# alexnet
# %cd over9000/
# try efficientnet later https://www.kaggle.com/khursani8/fast-ai-ootb-cutout-efficientnet
# from over9000 import *
# model.unfreeze()
from timm import create_model

model = create_model('tf_efficientnet_b6', pretrained=True)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features=in_features, out_features=3, bias=True)
if torch.cuda.is_available():
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
model = model.to(device)

learn = Learner(data=data, model=model, loss_func = nn.CrossEntropyLoss())

In [ ]:
# model
saved_model_name = 'fastai-efficientnet-adam-TTA-oversampling'

In [ ]:
# model.load('fastai-densenet')
learn.summary()

In [ ]:
# https://github.com/fastai/course-v3/blob/master/nbs/dl1/lesson1-pets.ipynb
# learn.unfreeze()
# print(callbacks)
from fastai.callbacks import *


callbacks_all = [SaveModelCallback(learn, every='improvement', monitor='valid_loss', name = saved_model_name),
             EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.001, patience=7)]
             
learn.fit_one_cycle(40, 1e-4, callbacks=callbacks_all)

epoch,train_loss,valid_loss,time
0,1.083731,1.082413,03:24
1,1.076057,1.061423,02:24
2,1.063332,1.039357,02:24
3,1.040134,0.994370,02:20
4,1.003821,0.921057,02:19
5,0.946247,0.798776,02:23
6,0.839649,0.619747,02:21
7,0.705628,0.435428,02:19
8,0.565981,0.359388,02:17
9,0.452981,0.322273,02:17


Better model found at epoch 0 with valid_loss value: 1.0824127197265625.
Better model found at epoch 1 with valid_loss value: 1.061423420906067.
Better model found at epoch 2 with valid_loss value: 1.03935706615448.
Better model found at epoch 3 with valid_loss value: 0.9943698644638062.
Better model found at epoch 4 with valid_loss value: 0.9210569858551025.
Better model found at epoch 5 with valid_loss value: 0.7987760305404663.
Better model found at epoch 6 with valid_loss value: 0.6197468638420105.
Better model found at epoch 7 with valid_loss value: 0.4354279637336731.
Better model found at epoch 8 with valid_loss value: 0.35938772559165955.
Better model found at epoch 9 with valid_loss value: 0.32227298617362976.
Better model found at epoch 10 with valid_loss value: 0.29946961998939514.
Better model found at epoch 11 with valid_loss value: 0.28622201085090637.
Epoch 19: early stopping


In [ ]:
# learn = Learner(data=data, model=model, loss_func = nn.CrossEntropyLoss())
learn.load(saved_model_name)

In [ ]:
# preds_test, y_test = model.get_preds(DatasetType.Test)
preds_test,y = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
# preds_test, y = preds_test

In [ ]:
preds_test.shape

torch.Size([610, 3])

In [ ]:
import os
test_path = DATASET_PATH+"/test/test"
# test_path = PROJECT_PATH + "/other-data/test/"
# print(model.data.classes)
submission = pd.DataFrame({'ID': [s.split('.')[0] for s in os.listdir(test_path)  if s.split('.')[1] != 'jfif']})

for i, c in enumerate(learn.data.classes):
  # print(preds_test[:,i].shape)
  # print(c)
  submission[c] = preds_test[:,i]
submission = submission[['ID', 'leaf_rust', 'stem_rust', 'healthy_wheat']] # Get same order as sample sub
submission.head()

,ID,leaf_rust,stem_rust,healthy_wheat
0,HFV4BK,0.987835,0.004648,0.007517
1,0L5TC8,0.977792,0.015913,0.006295
2,VW34Z3,0.432817,0.555147,0.012037
3,HGSMHR,0.021502,0.966172,0.012327
4,0K9T9T,0.149461,0.758348,0.092191


In [ ]:
submission.to_csv(PROJECT_PATH+'/submissions/'+saved_model_name+'.csv', index = False)

In [ ]:
# !ls -l "{test_path}" | wc -l
# test_path = PROJECT_PATH + "/other-data/test/"
# len(os.listdir(test_path))
print(submission.shape)

(610, 4)


In [ ]:
print(learn.data.classes)